In [1]:
# Imports

import argparse
import time
import math
import os
import torch
import torch.nn as nn
import torch.onnx

import data
import model

In [2]:
if torch.cuda.is_available():
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
###############################################################################
# Load data
###############################################################################
batch_size = 20
corpus = data.Corpus('./data/raw/penn-treebank')

# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.
# These columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
# batch processing.

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)


path--- ./data/raw/penn-treebank/ptb.train.txt
path--- ./data/raw/penn-treebank/ptb.valid.txt
path--- ./data/raw/penn-treebank/ptb.test.txt


In [9]:
###############################################################################
# Build the model
###############################################################################
emsize = 200
nhid = 200
nlayers = 1
dropout = 0.5
tied = False

ntokens = len(corpus.dictionary)
model = model.RNNModel('LSTM', ntokens, emsize, nhid, nlayers, dropout, tied).to(device)

criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [10]:
###############################################################################
# Training code
###############################################################################

bptt = 35
clip = 0.25
log_interval = 10
epochs = 10

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


# get_batch subdivides the source data into chunks of length bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.

def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
            hidden = repackage_hidden(hidden)
    return total_loss / (len(data_source) - 1)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt, lr,
                elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

# Loop over epochs.
lr = 20
best_val_loss = None
save_file = 'models/test'

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(save_file, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')


| epoch   1 |    10/ 1327 batches | lr 20.00 | ms/batch 292.03 | loss  9.35 | ppl 11506.29
| epoch   1 |    20/ 1327 batches | lr 20.00 | ms/batch 311.42 | loss  7.47 | ppl  1749.53
| epoch   1 |    30/ 1327 batches | lr 20.00 | ms/batch 270.73 | loss  7.18 | ppl  1308.37
| epoch   1 |    40/ 1327 batches | lr 20.00 | ms/batch 257.42 | loss  7.10 | ppl  1207.21
| epoch   1 |    50/ 1327 batches | lr 20.00 | ms/batch 240.44 | loss  6.93 | ppl  1019.11
| epoch   1 |    60/ 1327 batches | lr 20.00 | ms/batch 235.84 | loss  6.90 | ppl   994.82
| epoch   1 |    70/ 1327 batches | lr 20.00 | ms/batch 288.03 | loss  6.72 | ppl   825.76
| epoch   1 |    80/ 1327 batches | lr 20.00 | ms/batch 267.78 | loss  6.72 | ppl   825.97
| epoch   1 |    90/ 1327 batches | lr 20.00 | ms/batch 252.49 | loss  6.61 | ppl   740.83
| epoch   1 |   100/ 1327 batches | lr 20.00 | ms/batch 252.71 | loss  6.49 | ppl   657.95
| epoch   1 |   110/ 1327 batches | lr 20.00 | ms/batch 268.49 | loss  6.63 | ppl   758.81